In [6]:
import pandas as pd
import io
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.formatting.rule import FormulaRule
from datetime import date, datetime

# 1. 設定與模擬資料 (Data Setup)
# 讀取使用者上傳的 CSV 內容 (模擬讀取)
# 實際運作時，這裡會使用您提供的檔案路徑
# 這裡我將手動構建一個 DataFrame 來模擬您的 '承辦中.csv' 資料結構，以便生成範例
data = {
    '買方': ['柯遠東', '顏鼎承', '藍祐綸', '方妍云', '謝采庭'],
    '賣方': ['趙志龍', '歐典侃', '楊勝凱', '葉又慈', '趙婉芳'],
    '縣市': ['台北', '台北', '新北', '台北', '新北'],
    '類型': ['一般', '自用', '一般', '一般', '自用'],
    '成交價': [18500000, 27500000, 15000000, 13600000, 29850000],
    '簽約日': ['2024-07-19', '2025-06-21', '2025-06-06', '2025-10-07', '2025-11-02'],
    '完稅日': ['2024-09-09', '2025-09-02', '2025-07-31', '2025-03-02', '2025-01-20'],
    '銀行': ['元大', '台新', '中信', '兆豐', '自找'],
    '備註': ['稅單好了 塗二胎', '訴訟中, 勿壓日期', '營登 騰空', '急件', '屋主用賴聯絡']
}
df_active = pd.DataFrame(data)

# 2. 初始化 Excel Workbook
wb = Workbook()

# --- A. 建立 Sheet: 【參數設定】 (Settings) ---
ws_settings = wb.active
ws_settings.title = "參數設定"
# 建立預設清單
locations = ["台北", "新北", "基隆", "桃園"]
types = ["一般", "自用", "換約", "地上權"]
banks = ["台新", "中信", "富邦", "國泰", "兆豐", "元大", "土銀", "第一"]
# 寫入 Excel
ws_settings['A1'] = "縣市清單"
ws_settings['B1'] = "案件類型"
ws_settings['C1'] = "銀行清單"
for i, item in enumerate(locations): ws_settings.cell(row=i+2, column=1, value=item)
for i, item in enumerate(types): ws_settings.cell(row=i+2, column=2, value=item)
for i, item in enumerate(banks): ws_settings.cell(row=i+2, column=3, value=item)
ws_settings.sheet_state = 'hidden' # 隱藏此頁

# --- B. 建立 Sheet: 【承辦中】 (Active Cases) ---
ws_active = wb.create_sheet("承辦中")

# 定義欄位架構 (Column Schema)
# Tuple 結構: (標題, 寬度, 樣式顏色Hex, 資料驗證類型)
# 顏色代碼: 淺藍=E1F5FE, 淺綠=E8F5E9, 黃橘=FFF3E0, 紫=F3E5F5
columns_config = [
    # A. 案件基本資料 (淺藍)
    ("案號", 10, "E1F5FE", None),
    ("買方", 12, "E1F5FE", None),
    ("賣方", 12, "E1F5FE", None),
    ("縣市", 8, "E1F5FE", "List_Location"),
    ("類型", 8, "E1F5FE", "List_Type"),
    ("成交價", 15, "E1F5FE", "Number"),
    
    # B. 財務與貸款 (淺綠)
    ("買方銀行", 12, "E8F5E9", "List_Bank"),
    ("核貸狀態", 12, "E8F5E9", None), # 輸入 OK 或 金額
    ("賣方銀行", 12, "E8F5E9", None),
    ("代償金額", 15, "E8F5E9", "Number"),
    
    # C. 關鍵時程與規費 (黃橘) - 核心甘特圖來源
    ("簽約日", 12, "FFF3E0", "Date"),
    ("用印日", 12, "FFF3E0", "Date"),
    ("預收規費_日", 12, "FFF3E0", "Date"), # 拆分
    ("預收規費_金", 10, "FFF3E0", "Number"), # 拆分
    ("完稅日", 12, "FFF3E0", "Date"),
    ("過戶日", 12, "FFF3E0", "Date"),
    ("交屋日", 12, "FFF3E0", "Date"),
    
    # D. 待辦事項矩陣 (Checklist Matrix) (預設灰底)
    # 使用短標題以便縮小欄寬
    ("買印", 5, "DDDDDD", "Boolean"),
    ("賣印", 5, "DDDDDD", "Boolean"),
    ("用印款", 5, "DDDDDD", "Boolean"),
    ("完稅款", 5, "DDDDDD", "Boolean"),
    ("權狀", 5, "DDDDDD", "Boolean"),
    ("規費", 5, "DDDDDD", "Boolean"),
    ("設定", 5, "DDDDDD", "Boolean"),
    ("稅單", 5, "DDDDDD", "Boolean"),
    ("差額", 5, "DDDDDD", "Boolean"),
    ("整過戶", 5, "DDDDDD", "Boolean"),

    # E. 風控與備註 (紫)
    ("檢核:退稅", 8, "F3E5F5", "Boolean_YN"),
    ("檢核:戶籍", 8, "F3E5F5", "Boolean_YN"),
    ("應注意事項", 30, "F3E5F5", None)
]

# 寫入標題與設定格式
header_font = Font(bold=True, name="微軟正黑體")
center_align = Alignment(horizontal='center', vertical='center')
thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))

for col_idx, (header, width, color, validation) in enumerate(columns_config, 1):
    col_letter = get_column_letter(col_idx)
    cell = ws_active.cell(row=1, column=col_idx, value=header)
    
    # 樣式設定
    cell.font = header_font
    cell.alignment = center_align
    cell.border = thin_border
    cell.fill = PatternFill(start_color=color, end_color=color, fill_type="solid")
    ws_active.column_dimensions[col_letter].width = width

# 寫入模擬資料 (從 DataFrame)
for row_idx, row_data in df_active.iterrows():
    excel_row = row_idx + 2
    # 對應資料填入 (這裡簡化對應，實際依您需求調整)
    ws_active.cell(row=excel_row, column=2, value=row_data['買方'])
    ws_active.cell(row=excel_row, column=3, value=row_data['賣方'])
    ws_active.cell(row=excel_row, column=4, value=row_data['縣市'])
    ws_active.cell(row=excel_row, column=5, value=row_data['類型'])
    ws_active.cell(row=excel_row, column=6, value=row_data['成交價'])
    ws_active.cell(row=excel_row, column=7, value=row_data['銀行'])
    
    # 日期資料
    ws_active.cell(row=excel_row, column=11, value=row_data['簽約日']) # 簽約
    ws_active.cell(row=excel_row, column=15, value=row_data['完稅日']) # 完稅
    
    ws_active.cell(row=excel_row, column=28, value=row_data['備註']) # 備註

# --- 設定資料驗證 (Data Validation) ---
# 1. 縣市下拉
dv_city = DataValidation(type="list", formula1="=參數設定!$A$2:$A$10", allow_blank=True)
ws_active.add_data_validation(dv_city)
dv_city.add(f"D2:D100")

# 2. 銀行下拉
dv_bank = DataValidation(type="list", formula1="=參數設定!$C$2:$C$20", allow_blank=True)
ws_active.add_data_validation(dv_bank)
dv_bank.add(f"G2:G100")

# 3. 矩陣檢核 (Checklist) - 使用 'v' 或 'OK'
# 我們可以設定輸入 'v' 就變綠色
# 這裡為了簡化，我們主要依賴條件格式化

# --- 設定條件格式化 (Conditional Formatting) ---
# 1. 矩陣視覺化: 當儲存格不為空 (有打勾或輸入v)，背景變綠
green_fill = PatternFill(start_color="C8E6C9", end_color="C8E6C9", fill_type="solid")
# 假設矩陣欄位是 R 到 AA (18-27)
matrix_range = "R2:AA100"
ws_active.conditional_formatting.add(matrix_range, FormulaRule(formula=['NOT(ISBLANK(R2))'], stopIfTrue=True, fill=green_fill))

# 2. 風險提示: 訴訟關鍵字
red_fill = PatternFill(start_color="FFCDD2", end_color="FFCDD2", fill_type="solid")
red_font = Font(color="B71C1C", bold=True)
# 檢查備註欄 (AC欄，即第29欄) 是否包含 "訴訟"
# 注意: 公式中的欄位需對應實際位置
ws_active.conditional_formatting.add("A2:AC100", FormulaRule(formula=['ISNUMBER(SEARCH("訴訟", $AC2))'], stopIfTrue=True, fill=red_fill))

# 3. 提醒立契日: 核貸OK 但 完稅日為空
# 假設核貸狀態是 H欄 (第8欄)，完稅日是 O欄 (第15欄)
purple_font = Font(color="4A148C", bold=True)
ws_active.conditional_formatting.add("H2:H100", FormulaRule(formula=['AND(H2="OK", ISBLANK($O2))'], stopIfTrue=True, font=purple_font))

# --- C. 建立 Sheet: 【結案歸檔】 (Archive) ---
ws_archive = wb.create_sheet("結案歸檔")
# 複製承辦中的標頭
for col_idx, (header, width, _, _) in enumerate(columns_config, 1):
    col_letter = get_column_letter(col_idx)
    ws_archive.cell(row=1, column=col_idx, value=header).font = header_font
    ws_archive.column_dimensions[col_letter].width = width
ws_archive.cell(row=1, column=len(columns_config)+1, value="結案日期") # 多加一欄

# --- D. 建立 Sheet: 【進度儀表板】 (Timeline/Gantt) ---
ws_gantt = wb.create_sheet("進度儀表板")
ws_gantt['A1'] = "此頁面未來可設定自動讀取「承辦中」資料並繪製甘特圖"
ws_gantt['A2'] = "建議使用條件格式化：如果 [日期] 介於 [簽約] 與 [交屋] 之間，則填色。"

# 存檔
file_path = '代書案件管理系統_v1.xlsx'
wb.save(file_path)
print(f"系統範本已生成: {file_path}")

系統範本已生成: 代書案件管理系統_v1.xlsx
